In [58]:
import pandas as pd
import numpy as np 
import csv as csv
import matplotlib.pyplot as plt
import random
import json

from scipy.stats import poisson

import geopandas as gpd
from shapely.geometry import Polygon, Point, MultiPolygon
from shapely.ops import unary_union
from pyproj import CRS
import re

from sklearn.cluster import DBSCAN

# Bevölkerungsdaten - Zensus Transformation

In [ ]:
file_path = './Einwohner_Data/Bevoelkerung100M.csv'
df = pd.read_csv(file_path, encoding='ISO-8859-1', delimiter=';')

In [ ]:
list_of_merkmale = [' INSGESAMT', 'ALTER_KURZ', 'GESCHLECHT', 'ALTER_10JG']
df_einwohner = df[df['Merkmal'].isin(list_of_merkmale)]

df_bezeichnungen = pd.DataFrame()
bezeichnungen = ['Merkmal', 'Auspraegung_Text', 'Auspraegung_Code']
df_bezeichnungen = df_einwohner[bezeichnungen]
df_bezeichnungen.drop_duplicates(inplace=True)
df_bezeichnungen.sort_values(by=['Merkmal', 'Auspraegung_Code'], inplace=True)
df_bezeichnungen.reset_index(drop=True, inplace=True)
df_bezeichnungen.to_csv('./Einwohner_Data/Bezeichnungen.csv', index=False)

df_einwohner['Attribute'] = df_einwohner['Merkmal'] + '_' + df_einwohner['Auspraegung_Code'].astype(str)
df_einwohner.drop(columns=['Merkmal', 'Auspraegung_Code', 'Auspraegung_Text', 'Gitter_ID_100m', 'Anzahl_q'], inplace=True)

In [ ]:
pivot_df = df_einwohner.pivot(index=['Gitter_ID_100m_neu'], columns=['Attribute'], values='Anzahl').reset_index()
pivot_df.columns.name = None
pivot_df = pivot_df.fillna(0)
pivot_df.rename(columns={' INSGESAMT_0': 'INSGESAMT_0'}, inplace=True)
pivot_df.to_csv('./Einwohner_Data/Einwohner_Cleaned.csv')

In [ ]:
df_geo = pivot_df
# Extrahiere Informationen aus der Gitter-IDs mithilfe von regulären Ausdrücken
pattern = re.compile(r'N(\d+)E(\d+)')
df_geo[['origin_n', 'origin_e']] = df_geo['Gitter_ID_100m_neu'].str.extract(pattern).astype(int)

# Setze feste Werte für crs_code und resolution 
df_geo['crs_code'] = 3035
df_geo['resolution'] = 100

# Erstelle Geometrien (Quadrate) für jedes Gitter
df_geo['geometry'] = [Polygon([(e, n), (e + 100, n), (e + 100, n - 100), (e, n - 100)]) for e, n in zip(df_geo['origin_e'], df_geo['origin_n'])]
df_geo.to_csv('./GeoDaten/GeoDataFrame_Bevoelkerung.csv')


In [ ]:
gdf = gpd.GeoDataFrame(df_geo, geometry='geometry', crs=CRS(f"EPSG:{df_geo['crs_code'].iloc[0]}"))

vmin, vmax = 0, 20

# Plotte das GeoDataFrame mit Farbhervorhebung der 'INSGESAMT'-Werte und angepasster Skala
gdf.plot(column='INSGESAMT_0', cmap='viridis', legend=True, figsize=(10, 10), vmin=vmin, vmax=vmax)
plt.title('Farbliche Hervorhebung der INSGESAMT-Werte')
plt.show()
gdf.to_file('./GeoDaten/Deutschland_Raster.gpkg', driver='GPKG')

# Laden der Verwaltungsbezirke um die Raster zu filtern

In [ ]:
geo_bundesländer = gpd.read_file('./GeoDaten/DE_NUTS5000.gpkg')

geo_würzburg = geo_bundesländer[geo_bundesländer['NUTS_NAME'].str.contains('Würzburg')]
geo_würzburg = geo_würzburg.to_crs(gdf.crs)
mask_overlapping = gdf.geometry.intersects(geo_würzburg.unary_union)
gdf_würzburg = gdf[mask_overlapping]
gdf_würzburg.to_file('./GeoDaten/Wuerzburg.gpkg', driver='GPKG')

# Gebäudedaten und Bevölkerungsdaten mappen

In [ ]:
df_buildings = pd.read_csv('./OSM_Data/Würzburg-Buildings.csv')
gdf_würzburg = gpd.read_file('./GeoDaten/Wuerzburg.gpkg')

## Gebäude einem Raster zuordnen

In [ ]:
def find_nearest_polygon_id(point, polygons):
    return polygons.distance(point.geometry).sort_values().index[0] 

In [ ]:
gdf_buildings = gpd.GeoDataFrame(df_buildings, geometry=gpd.points_from_xy(df_buildings['lon'], df_buildings['lat']), crs=CRS("EPSG:4326"))
gdf_buildings = gdf_buildings.to_crs(gdf_würzburg.crs)
df_buildings['Raster_ID'] = -1

for index, building in gdf_buildings.iterrows():

    mask_overlapping = gdf_würzburg.intersects(building['geometry'])
    
    if any(mask_overlapping):
        df_buildings.loc[index, 'Raster_ID'] = gdf_würzburg[mask_overlapping].index[0]
    else:
        df_buildings.loc[index, 'Raster_ID'] = find_nearest_polygon_id(building, gdf_würzburg)


gdf_buildings_with_raster = gpd.GeoDataFrame(df_buildings, geometry=gdf_buildings['geometry'], crs=gdf_würzburg.crs)

df_buildings.to_csv('./OSM_Data/Wuerzburg_Buildings_Raster.csv')
gdf_buildings_with_raster.to_file('./GeoDaten/Buildings_Raster.gpkg', driver='GPKG')

In [ ]:
gdf_buildings_with_raster = gpd.read_file('./GeoDaten/Buildings_Raster.gpkg')
gdf_würzburg = gpd.read_file('./GeoDaten/Wuerzburg.gpkg')

In [ ]:
# Zähle die eindeutigen Werte in der Spalte 'Spalte_Name'
value_counts = gdf_buildings_with_raster['Raster_ID'].value_counts()

# Sortiere die Ergebnisse nach den Werten
sorted_value_counts = value_counts.sort_values(ascending = False)

# Gib die sortierten Ergebnisse aus
print("Sortierte Wertezählungen:")
print(sorted_value_counts)

gdf_buildings_with_raster[gdf_buildings_with_raster['Raster_ID'] == 8207].plot()

## Jedem Gebäude die Bevölkerung zuweisen

In [ ]:
gdf_buildings_with_raster['Einwohner'] = 0
gdf_buildings_with_raster['Alter'] = {} 
gdf_buildings_with_raster['Geschlecht'] = {} 


unique_raster_indices = gdf_buildings_with_raster['Raster_ID'].unique()

for raster_index in unique_raster_indices:
    einwohner_aus_raster = gdf_würzburg.at[raster_index, 'INSGESAMT_0']

    alter_dict_raster = {
            '0-10':  int(gdf_würzburg.at[raster_index, 'ALTER_10JG_1']),
            '10-20': int(gdf_würzburg.at[raster_index, 'ALTER_10JG_2']),
            '20-30': int(gdf_würzburg.at[raster_index, 'ALTER_10JG_3']),
            '30-40': int(gdf_würzburg.at[raster_index, 'ALTER_10JG_4']), 
            '40-50': int(gdf_würzburg.at[raster_index, 'ALTER_10JG_5']), 
            '50-60': int(gdf_würzburg.at[raster_index, 'ALTER_10JG_6']),
            '60-70': int(gdf_würzburg.at[raster_index, 'ALTER_10JG_7']), 
            '70-80': int(gdf_würzburg.at[raster_index, 'ALTER_10JG_8']),
            '80+':   int(gdf_würzburg.at[raster_index, 'ALTER_10JG_9'])
        }
    
    geschlecht_dict_raster = {
            'maennlich': gdf_würzburg.at[raster_index, 'GESCHLECHT_1'],
            'weiblich':  gdf_würzburg.at[raster_index, 'GESCHLECHT_2']
        }
   
    for index, building in gdf_buildings_with_raster[gdf_buildings_with_raster['Raster_ID'] == raster_index].iterrows():

        alter_dict_geb = {
            '0-10': 0,
            '10-20': 0,
            '20-30': 0,
            '30-40': 0, 
            '40-50': 0, 
            '50-60': 0,
            '60-70': 0, 
            '70-80': 0,
            '80+': 0
        }

        geschlecht_dict_geb = {
            'maennlich': 0,
            'weiblich': 0
        }


        if index == gdf_buildings_with_raster[gdf_buildings_with_raster['Raster_ID'] == raster_index].index.max():
            random_einwohner = einwohner_aus_raster
        else:
            random_einwohner = np.random.choice(int(einwohner_aus_raster))

        
        random_einwohner = int(random_einwohner)
        alter_count = random_einwohner
        geschlecht_count = random_einwohner

        for key in np.random.permutation(list(alter_dict_raster.keys())):
            if alter_count == 0:
                break
            if alter_dict_raster[key] > 0:
                if alter_count <= alter_dict_raster[key]:
                    alter_dict_geb[key] = alter_count
                    alter_dict_raster[key] -= alter_count
                    alter_count = 0

                else:
                    alter_dict_geb[key] = alter_dict_raster[key]
                    alter_count -= alter_dict_raster[key]
                    alter_dict_raster[key] = 0
        

        if alter_count != 0:
            alter_dict_geb[random.choice(list(alter_dict_geb.keys()))] += alter_count
            


        for key in np.random.permutation(list(geschlecht_dict_raster.keys())):
            if geschlecht_count == 0:
                break
            if geschlecht_dict_raster[key] > 0:
                if geschlecht_count <= geschlecht_dict_raster[key]:
                    geschlecht_dict_geb[key] = geschlecht_count
                    geschlecht_dict_raster[key] -= geschlecht_count
                    geschlecht_count = 0

                else:
                    geschlecht_dict_geb[key] = geschlecht_dict_raster[key]
                    geschlecht_count -= geschlecht_dict_raster[key]
                    geschlecht_dict_raster[key] = 0


        if geschlecht_count != 0:
            geschlecht_dict_geb[random.choice(list(geschlecht_dict_geb.keys()))] += geschlecht_count


        gdf_buildings_with_raster.at[index, 'Einwohner'] = random_einwohner
        gdf_buildings_with_raster.loc[index, 'Alter'] = [alter_dict_geb]
        gdf_buildings_with_raster.loc[index, 'Geschlecht'] = [geschlecht_dict_geb]
        einwohner_aus_raster -= random_einwohner

In [ ]:
gdf_buildings_with_raster['Alter'] = gdf_buildings_with_raster['Alter'].apply(json.dumps)
gdf_buildings_with_raster['Geschlecht'] = gdf_buildings_with_raster['Geschlecht'].apply(json.dumps)

gdf_buildings_with_raster.to_file('./GeoDaten/Buildings_Raster_Demographie.gpkg', driver='GPKG')

## Jedem Einwohner einen eigenen Tupel erstellen

In [ ]:
gdf_einwohner = gpd.read_file('./GeoDaten/Buildings_Raster_Demographie.gpkg')

gdf_einwohner['Alter'] = gdf_einwohner['Alter'].apply(json.loads)
gdf_einwohner['Geschlecht'] = gdf_einwohner['Geschlecht'].apply(json.loads)

In [ ]:
# Neue leere Liste für die aufgeschlüsselten Daten
new_rows = []

# Iteration über jede Zeile des DataFrame
for index, row in gdf_einwohner.iterrows():
    # Iteration über die Altersgruppen in der Spalte 'Alter'
    
    maennlich = int(row['Geschlecht'][0].get('maennlich', 0))

    for age_group, count in row['Alter'][0].items():
        
        for i in range(count):

            # Erstellung einer neuen Zeile für jede Altersgruppe
            new_row = row.copy()
            new_row['Einwohner'] = 1  # Aktualisierung der Einwohnerzahl für die Altersgruppe
            new_row['Alter'] = age_group  # Aktualisierung der Altersgruppe
            if maennlich != 0:
                new_row['Geschlecht'] = 'maennlich'
                maennlich -= 1

            else:
                new_row['Geschlecht'] = 'weiblich'


            new_rows.append(new_row)

# Neuen DataFrame erstellen
new_df = pd.DataFrame(new_rows)


## Jedem Einwohner eine Nachfrage zuweisen

In [ ]:
gdf_loaded = gpd.read_file('./GeoDaten/Buildings_Raster_Demographie.gpkg')

gdf_loaded['Alter'] = gdf_loaded['Alter'].apply(json.loads)
gdf_loaded['Geschlecht'] = gdf_loaded['Geschlecht'].apply(json.loads)

In [ ]:
geo_bundesländer = gpd.read_file('./GeoDaten/DE_NUTS5000.gpkg')

geo_würzburg = geo_bundesländer[geo_bundesländer['NUTS_NAME'].str.contains('Würzburg')]
geo_würzburg = geo_würzburg.to_crs(gdf_würzburg.crs)

In [ ]:
# Wahrscheinlichkeitsverteilung
probabilities = {
    '0-10': 0.01,
    '10-20': 0.02,
    '20-30': 0.03,
    '30-40': 0.04,
    '40-50': 0.07,
    '50-60': 0.09,
    '60-70': 0.1,
    '70-80': 0.15,
    '80+': 0.2
}

In [ ]:
df_data = gdf_loaded

for index, row in df_data.iterrows():
    age_groups_list = row['Alter']

    if age_groups_list:
        age_groups_dict = age_groups_list[0]
        
        for age_group, probability in probabilities.items():
            if age_group in age_groups_dict:
                num_residents = age_groups_dict[age_group]
                num_selected = np.random.binomial(num_residents, probability)
                df_data.at[index, 'Nachfrage'] += num_selected
            else:
                print(f"Die Altersgruppe '{age_group}' ist nicht im Dictionary enthalten.")
    else:
        print("Die Liste 'Alter' ist leer.")

In [ ]:
df_data['Alter'] = gdf_buildings_with_raster['Alter'].apply(json.dumps)
df_data['Geschlecht'] = gdf_buildings_with_raster['Geschlecht'].apply(json.dumps)

df_data.to_file('./Nachfrage/Wuerzburg_Nachfrage.gpkg', driver='GPKG')

In [ ]:
value_counts = df_data['Nachfrage'].value_counts()
sorted_value_counts = value_counts.sort_values(ascending = False)

print("Sortierte Wertezählungen:")
print(sorted_value_counts)

In [ ]:
# Plot des GeoDataFrames mit hervorgehobener "Nachfrage"-Spalte
ax = df_data.plot(column='Nachfrage', cmap='viridis', legend=True, figsize=(4, 4))

# Legende anzeigen
plt.show()

# Haushalte-Daten Zensus Transformation

In [ ]:
file_path = './Einwohner_Data/Haushalte100m.csv'
df_haushalte = pd.read_csv(file_path, encoding='ISO-8859-1', delimiter=',')

In [ ]:
df_bezeichnungen = pd.DataFrame()
bezeichnungen = ['Merkmal', 'Auspraegung_Text', 'Auspraegung_Code']
df_bezeichnungen = df_haushalte[bezeichnungen]
df_bezeichnungen.drop_duplicates(inplace=True)
df_bezeichnungen.sort_values(by=['Merkmal', 'Auspraegung_Code'], inplace=True)
df_bezeichnungen.reset_index(drop=True, inplace=True)
df_bezeichnungen.to_csv('./Einwohner_Data/Bezeichnungen_Haushalte.csv', index=False)

In [ ]:
df_haushalte['Attribute'] = df_haushalte['Merkmal'] + '_' + df_haushalte['Auspraegung_Text'].astype(str)
df_haushalte.drop(columns=['Merkmal', 'Auspraegung_Code', 'Auspraegung_Text', 'Gitter_ID_100m', 'Anzahl_q'], inplace=True)

In [ ]:
pivot_df_haushalte = df_haushalte.pivot(index=['Gitter_ID_100m_neu'], columns=['Attribute'], values='Anzahl').reset_index()
pivot_df_haushalte.columns.name = None
pivot_df_haushalte = pivot_df_haushalte.fillna(0)
pivot_df_haushalte.to_csv('./Einwohner_Data/Haushalte_Cleaned.csv')

In [ ]:
df_geo = pivot_df_haushalte
# Extrahiere Informationen aus der Gitter-IDs mithilfe von regulären Ausdrücken
pattern = re.compile(r'N(\d+)E(\d+)')
df_geo[['origin_n', 'origin_e']] = df_geo['Gitter_ID_100m_neu'].str.extract(pattern).astype(int)

# Setze feste Werte für crs_code und resolution 
df_geo['crs_code'] = 3035
df_geo['resolution'] = 100

# Erstelle Geometrien (Quadrate) für jedes Gitter
df_geo['geometry'] = [Polygon([(e, n), (e + 100, n), (e + 100, n - 100), (e, n - 100)]) for e, n in zip(df_geo['origin_e'], df_geo['origin_n'])]
df_geo.to_csv('./GeoDaten/GeoDataFrame_Haushalte.csv')

In [ ]:
gdf_haushalte = gpd.GeoDataFrame(df_geo, geometry='geometry', crs=CRS(f"EPSG:{df_geo['crs_code'].iloc[0]}"))

vmin, vmax = 0, 20

# Plotte das GeoDataFrame mit Farbhervorhebung der 'INSGESAMT'-Werte und angepasster Skala
gdf_haushalte.plot(column='INSGESAMT_Einheiten insgesamt', cmap='viridis', legend=True, figsize=(10, 10), vmin=vmin, vmax=vmax)
plt.title('Farbliche Hervorhebung der INSGESAMT-Werte')
plt.show()
gdf_haushalte.to_file('./GeoDaten/Deutschland_Raster_Haushalte.gpkg', driver='GPKG')

# Nachfrage-Wahrscheinlichkeitsverteilung

## Nachfragestatistik - Apothekenbesuch und dauerhafte Einnahme

In [ ]:
# Frequent von Apothekenbesuchen nach Altersverteilung
"Mehrmals in der Woche, Etwa einmal in der Woche, Zwei- bis dreimal im Monat, Einmal im Monat, Etwa einmal im Vierteljahr, Seltener, Nie, so gut wie nie"
apotheken_besuch = {
    '14-19': [0.002, 0.005, 0.023, 0.053, 0.18, 0.305, 0.432],
    '20-29': [0.003, 0.009, 0.052, 0.131, 0.282, 0.326, 0.197],
    '30-39': [0.006, 0.013, 0.098, 0.198, 0.287, 0.275, 0.123],
    '40-49': [0.009, 0.014, 0.105, 0.21, 0.297, 0.258, 0.106],
    '50-59': [0.008, 0.019, 0.116, 0.252, 0.308, 0.209, 0.087],
    '60-69': [0.005, 0.037, 0.173, 0.277, 0.278, 0.159, 0.071],
    '70+':   [0.011, 0.065, 0.252, 0.336, 0.196, 0.064, 0.075]
}


# Dauerhafte Einnahme von Medikamenten
'keine, ein bis zwei, drei, vier, fünf oder mehr'
dauerhafte_einnahme = {
    'Maenner': [0.51, 0.24, 0.08, 0.06, 0.11],
    'Frauen':  [0.41, 0.35, 0.09, 0.06, 0.09],
    '18-29':   [0.66, 0.30, 0.02, 0.01, 0.01],
    '30-49':   [0.59, 0.31, 0.05, 0.02, 0.03],
    '50-69':   [0.37, 0.31, 0.12, 0.08, 0.12],
    '70+':     [0.22, 0.23, 0.16, 0.14, 0.25]
}


In [ ]:
def check_sum_one(data_dict):
    for key, values in data_dict.items():
        total = sum(values)
        if not np.isclose(total, 1):
            print(f"Die Summe der Werte für {key} beträgt nicht 1, sondern {total}")

In [ ]:
check_sum_one(apotheken_besuch)
check_sum_one(dauerhafte_einnahme)

## Wiederholte Nachfrage und Behandlungsdauer

In [ ]:
# Packungsgrößen N1, N2, N3
    # N1: 10 Tage
    # N2: 30 Tage
    # N3: 100 Tage

# Transportkapazität

## Drohnenlogistik Unternehmen und Drohnenparameter

In [ ]:
# 5 größten Drohnenlogistik Unternehmen:
    # Drone Delivery Canada
    # Amazon.com, Inc.
    # Matternet
    # DHL
    # Zipline International Inc.

### Drone Delivery Canada

In [ ]:
# Sparrow
#     Max Range 20 km
#     Max Speed 60 km/h
#     Max Payload 4 kg
#     MTOW 25 kg
#     Aircraft Type Rotorcraft
#     Powerplant 8 Electric Motors
#     Navigation GPS-based
#     Delivery Options Land - Drop Ship

In [ ]:
# Canary
#     Max Range 20 km
#     Max Speed 72 km/h
#     Max Payload 4.5 kg
#     MTOW 25 kg
#     Aircraft Type Rotorcraft
#     Powerplant 8 Electric Motors
#     Navigation GPS-based
#     Delivery Options Land - Drop Ship

### Amazon.com, Inc.

In [ ]:
# Prime Air MK27-2
#     Max Range 25 km
#     Max Speed 80 km/h
#     Max Payload 2.26 kg
#     MTOW 25 kg
#     Max Altitude 122 m
#     Weather resistent

In [ ]:
# Prime Air MK30
#     ?

### Matternet

In [ ]:
# M2 Drone 
    # Max Range 20 km
    # Max Speed 50 km/h
    # Max Payload 2 kg
    # Max Altitude 120 m
    # MTOW 9.5 kg
    # Retail Price 7,499.00$
    # Aircraft type Quadcopter

### DHL

### Zipline International Inc.

In [ ]:
# P1
#   Max Range 193 km
#   Max Speed 105 km/h


In [ ]:
# P2


# Warehouse Location Problem Data Prep

## Clustering der Rastereinheiten

In [ ]:
gdf_wuerzburg = gpd.read_file('./GeoDaten/Wuerzburg.gpkg')

In [ ]:
# Plot der Raster mit Polygone farblich darstellen
ax = gdf_wuerzburg.plot(cmap='viridis', legend=True, figsize=(5, 5))

# Hervorhebung der Rastergrenzen
gdf_wuerzburg.boundary.plot(ax=ax, color='black')

# Titel hinzufügen
plt.title('Bevölkerungsraster des Landkreis Würzburg')

# Achsenbeschriftungen hinzufügen
plt.xlabel('Breitengradkoordinate im CSR3035 Format')
plt.ylabel('Längengradkoordinate im CSR3035 Format')

plt.show()

In [ ]:
# Extrahieren der Polygon-Koordinaten als Features für das Clustering
X = np.array(gdf_wuerzburg.geometry.apply(lambda polygon: [polygon.centroid.x, polygon.centroid.y]).tolist())

# DBSCAN-Clustering durchführen
dbscan = DBSCAN(eps=100, min_samples=4)  # Anpassen von eps und min_samples je nach Bedarf
dbscan.fit(X)

# Fügen Sie die Cluster-Zuordnung als neue Spalte zum GeoDataFrame hinzu
gdf_wuerzburg['cluster'] = dbscan.labels_

# Plotten der Polygone mit Cluster-Färbung
ax = gdf_wuerzburg.plot(column='cluster', cmap='tab20', legend=True, figsize=(5, 5))
plt.title('Clustering der Polygone mit DBSCAN')
plt.xlabel('X-Achse')
plt.ylabel('Y-Achse')
plt.show()

In [ ]:
# Alle eindeutigen Cluster-Werte und ihre Anzahl der Polygone ausgeben
cluster_counts = gdf_wuerzburg['cluster'].value_counts().sort_index()
print("Cluster-Werte und ihre Anzahl der Polygone:")
for cluster, count in cluster_counts.items():
    print(f"Cluster {cluster}: {count} Polygone")

In [ ]:
# # Extrahieren der eindeutigen Cluster-Bezeichnungen
# unique_clusters = gdf_wuerzburg['cluster'].unique()

# # Plotten der Polygone für jedes Cluster separat
# for cluster_label in unique_clusters:
#     if cluster_label == -1:  # Auslassung von Rauschpunkten (Clusterlabel -1)
#         continue
#     cluster_gdf = gdf_wuerzburg[gdf_wuerzburg['cluster'] == cluster_label]
#     ax = cluster_gdf.plot(color='blue', alpha=0.5, figsize=(10, 10))
#     plt.title(f'Cluster {cluster_label}')
#     plt.xlabel('X-Achse')
#     plt.ylabel('Y-Achse')
#     plt.show()

In [ ]:
# Plotten der Rauschpunkte (Clusterlabel -1)
noise_points = gdf_wuerzburg[gdf_wuerzburg['cluster'] == -1]
ax = noise_points.boundary.plot(color='red', alpha=0.5, figsize=(5, 5))
plt.title('Noise')
plt.xlabel('X-Achse')
plt.ylabel('Y-Achse')
plt.show()

In [ ]:
column_to_sum = 'INSGESAMT_0'
sum_of_column = noise_points[column_to_sum].sum()

print('Anzahl der Raster, die als Noise erkannt wurden: {}'.format(len(noise_points)))
print("Sum of column '{}' in noise points: {}".format(column_to_sum, sum_of_column))

In [ ]:
# Gruppierung nach dem Cluster und Aggregierung der Punkte in jedem Cluster
cluster_summary = gdf_wuerzburg.groupby('cluster').agg(
    Gitter_ID_100m_neu=('Gitter_ID_100m_neu', 'first'),
    count=('cluster', 'size'),  # Anzahl der Polygone in jedem Cluster
    sum_INSGESAMT_0=('INSGESAMT_0', 'sum'),  
    sum_ALTER_10JG_1=('ALTER_10JG_1', 'sum'), 
    sum_ALTER_10JG_2=('ALTER_10JG_2', 'sum'), 
    sum_ALTER_10JG_3=('ALTER_10JG_3', 'sum'), 
    sum_ALTER_10JG_4=('ALTER_10JG_4', 'sum'), 
    sum_ALTER_10JG_5=('ALTER_10JG_5', 'sum'), 
    sum_ALTER_10JG_6=('ALTER_10JG_6', 'sum'), 
    sum_ALTER_10JG_7=('ALTER_10JG_7', 'sum'), 
    sum_ALTER_10JG_8=('ALTER_10JG_8', 'sum'), 
    sum_ALTER_10JG_9=('ALTER_10JG_9', 'sum'),
    sum_GESCHLECHT_1=('GESCHLECHT_1', 'sum'),
    sum_GESCHLECHT_2=('GESCHLECHT_2', 'sum'),
    crs_code=('crs_code', 'first'),
    resolution=('resolution', 'first'),
    origin_n=('origin_n', 'first'),
    origin_e=('origin_e', 'first'),
    geometry=('geometry', lambda x: unary_union(x))  # Vereinigung der Polygone in jedem Cluster
) 

# Umwandeln des aggregierten DataFrames in ein GeoDataFrame
cluster_summary_gdf = gpd.GeoDataFrame(cluster_summary, geometry='geometry', crs=gdf_wuerzburg.crs)
cluster_summary_gdf = cluster_summary_gdf.reset_index(drop=False)

In [ ]:
# Zähler für Multipolygone initialisieren
multipolygon_count = 0

# Schleife über die Geometrien in der Spalte 'geometry_column'
for geometry in cluster_summary_gdf['geometry']:
    # Überprüfen, ob die Geometrie ein Multipolygon ist
    if isinstance(geometry, MultiPolygon):
        multipolygon_count += 1

# Anzeigen der Anzahl der Multipolygone
print("Anzahl der Multipolygone:", multipolygon_count)

In [ ]:
# Funktion zur Berechnung der Entfernung zwischen zwei Punkten
def distance(point1, point2):
    return point1.distance(point2)

cluster_summary_gdf['centroid'] = cluster_summary_gdf['geometry'].centroid

# Berechnung der durchschnittlichen Entfernung von jedem Polygon-Centroid zu seinen Ecken
average_distances = []
for idx, row in cluster_summary_gdf.iterrows():
    centroid = row['centroid']
    polygon = row['geometry']

    if isinstance(polygon, MultiPolygon):
        average_distance = 0  # Setzen Sie die durchschnittliche Entfernung auf 0 für Multipolygone
    else:
        # Extrahieren der Koordinaten der Polygon-Ecken
        polygon_corners = polygon.exterior.coords[:-1]  # Letzter Punkt ist der gleiche wie der erste
        # Berechnung der Entfernungen und Speichern der Ergebnisse
        distances = [distance(centroid, Point(coord)) for coord in polygon_corners]
        average_distance = sum(distances) / len(distances)
        
    average_distances.append(average_distance)

# Hinzufügen der durchschnittlichen Entfernungen als neue Spalte zum GeoDataFrame
cluster_summary_gdf['average_distance'] = average_distances

In [ ]:
# Plot der Raster mit Polygone farblich darstellen
ax = cluster_summary_gdf.plot(cmap='viridis', legend=True, figsize=(5, 5))

# Hervorhebung der Rastergrenzen
cluster_summary_gdf.boundary.plot(ax=ax, color='black')

# Titel hinzufügen
plt.title('Bevölkerungsraster des Landkreis Würzburg')

# Achsenbeschriftungen hinzufügen
plt.xlabel('Breitengradkoordinate im CSR3035 Format')
plt.ylabel('Längengradkoordinate im CSR3035 Format')

plt.show()

In [ ]:
# Centroid Geopandas speichern
save_gdf = cluster_summary_gdf
save_gdf = save_gdf.drop(columns=['geometry'])
save_gdf = save_gdf.rename(columns={'centroid': 'geometry'})
save_gdf.to_file('./WLP/centroid_wuerzburg.gpkg', driver='GPKG')

# Polygon Geopandas speichern
cluster_summary_gdf = cluster_summary_gdf.drop(columns=['centroid'])
cluster_summary_gdf = cluster_summary_gdf[cluster_summary_gdf['cluster'] != -1].reset_index(drop=True)
cluster_summary_gdf.to_file('./WLP/cluster_wuerzburg.gpkg', driver='GPKG')

## Mapping der Nachfrage auf die Cluster

In [ ]:
# Anhand der Packungsgrößen berechnet (N1 - N3), aufsummiert auf ein Jahr
# 2022 1.405 Mio. Packungen abgegeben

In [103]:
# Random Nachfrage von 100-1000 je Cluster um Daten zu simulieren
nachfrage_cluster = gpd.read_file('./WLP/centroid_wuerzburg.gpkg')

In [104]:
age_group_columns = [col for col in nachfrage_cluster.columns if col.startswith('sum_ALTER_10JG_')]

new_age_groups = { '0-10': 0,
                    '10-20': 3,
                    '20-30': 0,
                    '30-40': 0,
                    '40-50': 6,
                    '50-60': 8,
                    '60-70': 0,
                    '70-80': 9,
                    '80+': 0}

age_groups_list = list(new_age_groups.keys())


def get_age_group_data(nachfrage_cluster):
    age_group_data = {}

    for count in range(9):
        age_group =  age_groups_list[count] # Extract age group from column name
        population = nachfrage_cluster[age_group_columns[count]]
        age_group_data[age_group] = population
    return age_group_data


# Extract and merge age group data
age_group_data_per_cluster = nachfrage_cluster.apply(get_age_group_data, axis=1)

nachfrage_cluster['Alter'] = age_group_data_per_cluster

# Remove the original 'ALTER_10JG_' columns
nachfrage_cluster.drop(columns=age_group_columns, inplace=True)

In [105]:
# Assuming 'nachfrage_cluster' is your GeoPandas DataFrame
probabilities = {
    '0-10': 0.01,
    '10-20': 0.02,
    '20-30': 0.03,
    '30-40': 0.04,
    '40-50': 0.07,
    '50-60': 0.09,
    '60-70': 0.1,
    '70-80': 0.15,
    '80+': 0.2
}

def calculate_demand(row):
    total_demand = 0
    for age_group, count in row['Alter'].items():
        lambda_value = count * probabilities[age_group]
        total_demand += poisson.rvs(lambda_value)
    return total_demand * 365

nachfrage_cluster['nachfrage'] = nachfrage_cluster.apply(calculate_demand, axis=1)

In [106]:
# Berechnung des Gesamtbedarfs ohne die Noise-Punkte
total_demand = nachfrage_cluster[nachfrage_cluster['cluster'] != -1]['nachfrage'].sum()

# Summe der Polygone in jedem Cluster (ohne Noise)
bevoelkerung_sum = nachfrage_cluster[nachfrage_cluster['cluster'] != -1].groupby('cluster')['sum_INSGESAMT_0'].sum()

# Wert der Reihe mit den Noise-Punkten
noise_demand = nachfrage_cluster[nachfrage_cluster['cluster'] == -1]['nachfrage'].iloc[0]

# Verteilung des Noise-Bedarfs auf die anderen Cluster anteilig zur Anzahl der Polygone
for cluster in bevoelkerung_sum.index:
    cluster_demand = noise_demand * (bevoelkerung_sum[cluster] / bevoelkerung_sum.sum())
    nachfrage_cluster.loc[nachfrage_cluster['cluster'] == cluster, 'nachfrage'] += np.round(cluster_demand)

# Entfernen der Reihe mit dem 'cluster'-Wert von -1
nachfrage_cluster = nachfrage_cluster[nachfrage_cluster['cluster'] != -1].reset_index(drop=True)

In [113]:
nachfrage_cluster.to_file('./WLP/nachfrage_wuerzburg.gpkg', driver ='GPKG')